# 법률 문서 임베딩 및 Supabase 저장

- **임베딩 모델**: BGE-M3 (BAAI/bge-m3)
- **청킹**: 1024 tokens, 256 overlap
- **벡터 DB**: Supabase (pgvector + HNSW)

## 1. 환경 설정

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

if not SUPABASE_URL or not SUPABASE_KEY:
    raise ValueError("SUPABASE_URL과 SUPABASE_KEY를 .env 파일에 설정하세요")

print(f"Supabase URL: {SUPABASE_URL[:30]}...")

Supabase URL: https://elorfxocalhaymchopgp.s...


In [2]:
# 필요시 추가 설치
# !pip install FlagEmbedding python-dotenv

## 2. 문서 로드

In [3]:
DOCS_DIR = Path("docs")

documents = []
for md_file in sorted(DOCS_DIR.glob("*.md")):
    content = md_file.read_text(encoding="utf-8")
    documents.append({
        "filename": md_file.name,
        "content": content
    })
    print(f"로드: {md_file.name} ({len(content):,} chars)")

print(f"\n총 {len(documents)}개 문서 로드 완료")

로드: 1_자본시장과_금융투자업에_관한_법률.md (1,964 chars)
로드: 2_금융소비자_보호에_관한_법률_(금융소비자보호법_금소법).md (2,842 chars)
로드: 3_전자금융거래법.md (978 chars)
로드: 4_개인정보_보호법_신용정보의_이용_및_보호에_관한_법률.md (1,374 chars)

총 4개 문서 로드 완료


## 3. 청킹 (1024 / 256 overlap)

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=256,
    length_function=len,
    separators=["\n## ", "\n### ", "\n\n", "\n", " ", ""]
)

chunks = []
for doc in documents:
    doc_chunks = text_splitter.split_text(doc["content"])
    for i, chunk in enumerate(doc_chunks):
        chunks.append({
            "content": chunk,
            "metadata": {
                "filename": doc["filename"],
                "chunk_index": i
            }
        })
    print(f"{doc['filename']}: {len(doc_chunks)}개 청크")

print(f"\n총 {len(chunks)}개 청크 생성")

1_자본시장과_금융투자업에_관한_법률.md: 3개 청크
2_금융소비자_보호에_관한_법률_(금융소비자보호법_금소법).md: 5개 청크
3_전자금융거래법.md: 1개 청크
4_개인정보_보호법_신용정보의_이용_및_보호에_관한_법률.md: 2개 청크

총 11개 청크 생성


In [5]:
# 청크 샘플 확인
print(f"첫 번째 청크 ({len(chunks[0]['content'])} chars):")
print("-" * 50)
print(chunks[0]["content"][:500])
print("...")

첫 번째 청크 (915 chars):
--------------------------------------------------
# 자본시장과 금융투자업에 관한 법률 (자본시장법)

## 핵심 조문 - 금융투자업자 의무

### 제37조 (신의성실의무 등)

```
① 금융투자업자는 신의성실의 원칙에 따라 공정하게 금융투자업을 영위하여야 한다.
② 금융투자업자는 금융투자업을 영위함에 있어서 정당한 사유 없이 투자자의 이익을
   해하면서 자기가 이익을 얻거나 제삼자가 이익을 얻도록 하여서는 아니 된다.

```

### 제44조 (이해상충의 관리)

```
금융투자업자는 금융투자업의 영위와 관련하여 금융투자업자와 투자자 간,
특정 투자자와 다른 투자자 간의 이해상충을 방지하기 위하여
이해상충이 발생할 가능성을 파악·평가하고, 내부통제기준이 정하는 방법 및
절차에 따라 이를 적절히 관리하여야 한다.

```

### 제45조 (정보교류의 차단)

```
금융투자업자는 금융투자업의 영위와 관련하여 이해상충이 발생할 가능성이 있다고
인정되는 경우 금융투자업자의 부문 간 또는 금융투자업자와 그 계열회사 간의
정보교류
...


## 4. BGE-M3 임베딩 생성

In [6]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel(
    "BAAI/bge-m3",
    use_fp16=True  # GPU 메모리 절약
)

print("BGE-M3 모델 로드 완료")

/home/e7217/projects/FinAgent_Investment_Agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BGE-M3 모델 로드 완료


In [7]:
# 임베딩 생성
texts = [chunk["content"] for chunk in chunks]

embeddings = model.encode(
    texts,
    batch_size=8,
    max_length=1024
)["dense_vecs"]

print(f"임베딩 생성 완료: {embeddings.shape}")
print(f"임베딩 차원: {embeddings.shape[1]}")

pre tokenize: 100%|██████████| 2/2 [00:00<00:00, 675.30it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 2/2 [00:00<00:00, 30.68it/s]

임베딩 생성 완료: (11, 1024)
임베딩 차원: 1024


## 5. Supabase 테이블 생성

Supabase SQL Editor에서 아래 쿼리를 실행하세요:

In [8]:
create_table_sql = """
-- pgvector 확장 활성화
create extension if not exists vector;

-- 기존 테이블 삭제 (필요시)
-- drop table if exists law_documents;

-- 테이블 생성
create table law_documents (
  id bigserial primary key,
  content text not null,
  metadata jsonb,
  embedding vector(1024)
);

-- HNSW 인덱스 생성 (cosine 유사도)
create index law_documents_embedding_idx 
on law_documents 
using hnsw (embedding vector_cosine_ops)
with (m = 16, ef_construction = 64);

-- 유사도 검색 함수
create or replace function match_law_documents (
  query_embedding vector(1024),
  match_threshold float default 0.7,
  match_count int default 5
)
returns table (
  id bigint,
  content text,
  metadata jsonb,
  similarity float
)
language sql stable
as $$
  select
    id,
    content,
    metadata,
    1 - (embedding <=> query_embedding) as similarity
  from law_documents
  where 1 - (embedding <=> query_embedding) > match_threshold
  order by embedding <=> query_embedding
  limit match_count;
$$;
"""

print(create_table_sql)


-- pgvector 확장 활성화
create extension if not exists vector;

-- 기존 테이블 삭제 (필요시)
-- drop table if exists law_documents;

-- 테이블 생성
create table law_documents (
  id bigserial primary key,
  content text not null,
  metadata jsonb,
  embedding vector(1024)
);

-- HNSW 인덱스 생성 (cosine 유사도)
create index law_documents_embedding_idx 
on law_documents 
using hnsw (embedding vector_cosine_ops)
with (m = 16, ef_construction = 64);

-- 유사도 검색 함수
create or replace function match_law_documents (
  query_embedding vector(1024),
  match_threshold float default 0.7,
  match_count int default 5
)
returns table (
  id bigint,
  content text,
  metadata jsonb,
  similarity float
)
language sql stable
as $$
  select
    id,
    content,
    metadata,
    1 - (embedding <=> query_embedding) as similarity
  from law_documents
  where 1 - (embedding <=> query_embedding) > match_threshold
  order by embedding <=> query_embedding
  limit match_count;
$$;



## 6. Supabase에 임베딩 삽입

In [9]:
from supabase import create_client
import json

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

print("Supabase 클라이언트 연결 완료")

Supabase 클라이언트 연결 완료


In [11]:
# 데이터 삽입
records = []
for i, chunk in enumerate(chunks):
    records.append({
        "content": chunk["content"],
        "metadata": chunk["metadata"],
        "embedding": embeddings[i].tolist()
    })

# 배치 삽입
response = supabase.table("law_documents").insert(records).execute()

print(f"{len(records)}개 레코드 삽입 완료")

11개 레코드 삽입 완료


## 7. 검색 테스트

In [12]:
def search_laws(query: str, top_k: int = 3, threshold: float = 0.5):
    """법률 문서 유사도 검색"""
    # 쿼리 임베딩
    query_embedding = model.encode([query])["dense_vecs"][0].tolist()
    
    # Supabase RPC 호출
    response = supabase.rpc(
        "match_law_documents",
        {
            "query_embedding": query_embedding,
            "match_threshold": threshold,
            "match_count": top_k
        }
    ).execute()
    
    return response.data

In [13]:
# 테스트 쿼리
test_queries = [
    "투자자 보호 의무는 무엇인가?",
    "적합성 원칙이란?",
    "개인정보 수집 동의"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"쿼리: {query}")
    print("=" * 60)
    
    results = search_laws(query, top_k=2)
    
    for i, result in enumerate(results, 1):
        print(f"\n[{i}] 유사도: {result['similarity']:.4f}")
        print(f"    출처: {result['metadata']['filename']}")
        print(f"    내용: {result['content'][:200]}...")


쿼리: 투자자 보호 의무는 무엇인가?

[1] 유사도: 0.6251
    출처: 1_자본시장과_금융투자업에_관한_법률.md
    내용: # 자본시장과 금융투자업에 관한 법률 (자본시장법)

## 핵심 조문 - 금융투자업자 의무

### 제37조 (신의성실의무 등)

```
① 금융투자업자는 신의성실의 원칙에 따라 공정하게 금융투자업을 영위하여야 한다.
② 금융투자업자는 금융투자업을 영위함에 있어서 정당한 사유 없이 투자자의 이익을
   해하면서 자기가 이익을 얻거나 제삼자가 이익을 얻도록 ...

[2] 유사도: 0.5960
    출처: 1_자본시장과_금융투자업에_관한_법률.md
    내용: ## 핵심 조문 - 손해배상책임

### 제48조 (손해배상책임)

```
① 금융투자업자는 금융소비자 보호에 관한 법률 제19조 제1항 또는 제3항을
   위반한 경우 이로 인하여 발생한 일반투자자의 손해를 배상할 책임이 있다.
② 제1항에 따른 손해배상책임에 관하여는 「금융소비자 보호에 관한 법률」
   제44조를 적용한다.

```

### 제64조 ...

쿼리: 적합성 원칙이란?

[1] 유사도: 0.5546
    출처: 2_금융소비자_보호에_관한_법률_(금융소비자보호법_금소법).md
    내용: ## 6대 판매원칙

### 제17조 (적합성 원칙)

```
① 금융상품판매업자등은 금융상품계약체결등을 하거나 자문업무를 하는 경우에는
   상대방인 금융소비자가 일반금융소비자인지 전문금융소비자인지를 확인하여야 한다.

② 금융상품판매업자등은 일반금융소비자에게 다음 각 호의 금융상품 계약 체결을
   권유하는 경우에는 면담·질문 등을 통하여 다음 각 호의...

쿼리: 개인정보 수집 동의

[1] 유사도: 0.6031
    출처: 4_개인정보_보호법_신용정보의_이용_및_보호에_관한_법률.md
    내용: # 개인정보 보호법 / 신용정보의 이용 및 보호에 관한 법률

## 개인정보 보호법 핵심 내용

### 제15조 (개인정보의 수집·이용)

```
개인